In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
from functools import partial
import sklearn as sk
from  sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
train_x=sio.loadmat('E:\\Biyun Yan\\wpi\machine learning in cyber scurity\\pro5\\data\\Training_data_40classes_Tor_Browser.mat')
train_data=train_x['X_train3']
train_y=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Training_label_40classes_Tor_Browser.mat')
train_label=train_y['Ytrain'].T-1

In [3]:
test_x=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Test_data_40classes_Tor_Browser.mat')
test_data=test_x['X_test']
test_y=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Test_label_40classes_Tor_Browser.mat')
test_label=test_y['Ytest'].T-1

In [21]:
n_inputs=6000
n_hidden1=2000
n_hidden2=1000
n_hidden3=2000
n_outputs=n_inputs
learning_rate=0.01
l2_reg=0.0001

In [22]:
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None,1])
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
my_dense_layer = partial(tf.layers.dense,
                         activation=tf.nn.elu,
                         kernel_initializer=he_init,
                         kernel_regularizer=l2_regularizer)

hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)  # codings
hidden3 = my_dense_layer(hidden2, n_hidden3)
outputs = my_dense_layer(hidden3, n_outputs, activation=None)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))  # MSE



In [23]:
#svm_clf = svm.SVC()  # class 
#svm_fit=svm_clf.fit(train_outputs, train_label)
#logits=tf.layers.dense(outputs,40)

In [24]:
#loss_label=tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(Y,40),logits=logits)
#mean_loss=tf.reduce_mean(loss_label)
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)
#predicted_label=tf.arg_max(logits,1)
init = tf.global_variables_initializer()

In [25]:
#saver=tf.train.Saver()
with tf.Session() as sess:
    init.run()
    #max_acc=0
    for step in range(100):
        sess.run(training_op,feed_dict={X:train_data})
        #acc=np.sum(predicted_label_train==train_label)/train_data.shape[0]
        #print(mean_loss_train)
        #if max_acc<acc:
            #max_acc=acc
            #print('saving model')
           # saver.save(sess,'model_auto/')
    outs_train=sess.run(outputs,feed_dict={X:train_data})
    outs_test=sess.run(outputs,feed_dict={X:test_data})

In [26]:
svm_clf = svm.SVC()  # class 
svm_fit=svm_clf.fit(outs_train, train_label)
predict=svm_clf.predict(outs_test)
print(accuracy_score(test_label,predict))

C:\Users\motto\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\motto\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.335


In [15]:
with tf.Session() as sess:
    saver.restore(sess,'model_auto/')
    predicted_labels_test=sess.run(predicted_label,feed_dict={X:test_data,Y:test_label})
    acc=np.sum(predicted_labels_test==test_label)/test_data.shape[0]
    print(acc)

NameError: name 'saver' is not defined